In [1]:
from project_fraud.lib import drop_many_missing_values
from project_fraud.feature_engineering import clean_mail, make_day_feature, make_hour_feature, string_card, credit_cards, dist_from_mean, dist_from_median_rel


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import seaborn as sns


from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression

In [2]:
# transform_raw_data

In [3]:
def transform_raw_data():
    data = drop_many_missing_values()
    data['P_emaildomain_bin'], data['P_emaildomain_suffix'] = clean_mail(data)
    data['weekday'] = make_day_feature(data, offset=0.58)
    data['hours'] = make_hour_feature(data)
    data['cardID'] = data.apply(lambda row: string_card(row), axis=1)
    data = data.merge(credit_cards(data), how='left', on="cardID")
    data['dist_mean'] = data.apply(lambda row: dist_from_mean(row, 'mean'), axis=1)
    data['dist_median'] = data.apply(lambda row: dist_from_mean(row, 'median'), axis=1)
    data['dist_mean_rel'] = data.apply(lambda row: dist_from_median_rel(row, 'mean'), axis=1)
    data['dist_median_rel'] = data.apply(lambda row: dist_from_median_rel(row, 'median'), axis=1)
    return data

In [4]:
data_transformed = transform_raw_data()

In [37]:
data_transformed.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,hours,cardID,mean,min,max,median,dist_mean,dist_median,dist_mean_rel,dist_median_rel
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,0.0,2755404.0150.0mastercard102.0credit,235.020796,10.00,6085.23,115.00,-206.020796,-86.00,-0.876607,-0.747826
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,0.0,4663490.0150.0visa166.0debit,96.791005,12.50,994.00,59.00,-37.791005,0.00,-0.390439,0.000000
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,0.0,18132567.0150.0mastercard117.0debit,123.308485,6.00,3190.00,59.95,-73.308485,-9.95,-0.594513,-0.165972
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,4497514.0150.0mastercard102.0credit,96.972222,20.95,200.00,108.95,-46.972222,-58.95,-0.484388,-0.541074


In [39]:
data_transformed.P_emaildomain_suffix

0         nan
1         com
2         com
3         com
4         com
         ... 
590535    nan
590536    com
590537    com
590538    com
590539    com
Name: P_emaildomain_suffix, Length: 590540, dtype: object

In [36]:
type(data_transformed.P_emaildomain_suffix[590535])

str

In [40]:
def clean_mail_suffix(P_emaildomain_suffix):
    if P_emaildomain_suffix == 'nan':
        return np.nan
    else:
        return P_emaildomain_suffix

In [42]:
data_transformed['P_emaildomain_suffix'] = data_transformed['P_emaildomain_suffix'].apply(lambda x: clean_mail_suffix(x))

In [44]:
# Pipeline

In [45]:
X = data_transformed[['TransactionID','P_emaildomain_suffix','P_emaildomain_bin',
'card1','card2','addr1','TransactionAmt','card5','D15','C13','D2','D10','D4','weekday','hours',\
           'dist_mean', 'dist_median','dist_mean_rel','dist_median_rel']]
y = data_transformed['isFraud']

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.00017, random_state=1)

In [47]:
X_val.head(20)

,TransactionID,P_emaildomain_suffix,P_emaildomain_bin,card1,card2,addr1,TransactionAmt,card5,D15,C13,D2,D10,D4,weekday,hours,dist_mean,dist_median,dist_mean_rel,dist_median_rel
27047,3014047,com,apple,17188,321.0,299.0,34.50,226.0,377.0,8.0,226.0,377.0,377.0,0.0,21.0,-95.024776,-48.450,-0.733642,-0.584087
94624,3081624,com,yahoo,16783,282.0,428.0,171.00,226.0,265.0,18.0,NaN,265.0,328.0,1.0,15.0,-26.136727,54.000,-0.132582,0.461538
502930,3489930,com,microsoft,17404,581.0,325.0,129.95,226.0,619.0,3.0,530.0,619.0,489.0,5.0,15.0,-25.624182,32.000,-0.164707,0.326697
369483,3356483,com,google,16691,170.0,315.0,49.00,102.0,0.0,1.0,NaN,0.0,0.0,1.0,11.0,-109.738758,-43.000,-0.691317,-0.467391
26770,3013770,com,google,16116,568.0,325.0,34.00,226.0,0.0,1.0,NaN,0.0,0.0,0.0,20.0,-85.440484,-25.000,-0.715339,-0.423729
4591,2991591,com,google,6053,122.0,441.0,186.00,195.0,NaN,1.0,NaN,0.0,NaN,1.0,22.0,-21.360963,109.000,-0.103013,1.415584
278323,3265323,com,google,11839,490.0,123.0,57.95,226.0,0.0,71.0,14.0,103.0,490.0,1.0,13.0,-71.868382,-17.050,-0.553607,-0.227333
93031,3080031,com,google,8320,476.0,485.0,30.95,166.0,34.0,4.0,25.0,34.0,34.0,0.0,4.0,-70.636848,-28.050,-0.695335,-0.475424
93390,3080390,com,google,6062,159.0,325.0,100.00,117.0,NaN,1.0,NaN,NaN,NaN,1.0,11.0,-103.890576,11.050,-0.509541,0.124227
469816,3456816,com,yahoo,13249,111.0,264.0,226.00,226.0,41.0,19.0,6.0,620.0,502.0,6.0,3.0,106.021353,155.913,0.883669,2.224564


In [49]:
type(X_val['P_emaildomain_suffix'][390381])

float

In [50]:
X_val.to_csv('val_data.csv', index =False, na_rep='nan')

In [51]:
n = (X.dtypes != 'object')
num_cols = list(n[n].index)
medium_missing_num_cols = []
low_missing_num_cols =[]
for i in num_cols:
    percentage = data_transformed[i].isnull().sum() * 100 / len(data_transformed[i])
    if percentage < 15:
        low_missing_num_cols.append(i)
    elif percentage >= 15 and percentage <= 60:
        medium_missing_num_cols.append(i)

In [52]:
# trainer.py

num_transformer_low = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
num_transformer_medium = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('one_hot', OneHotEncoder())
])
    
preprocessor = ColumnTransformer([
    ('low_num_imputer',num_transformer_low, low_missing_num_cols),
    ('medium_num_imputer', num_transformer_medium, medium_missing_num_cols),
    ('cat_transformer', cat_pipeline, ['P_emaildomain_suffix','P_emaildomain_bin','weekday','hours'])],
    remainder='drop')

In [53]:
final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('log_reg', LogisticRegression(class_weight ='balanced'))])

In [54]:
final_pipe.fit(X_train, y_train)

/Users/gillesfaltz/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('preprocessing', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('low_num_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 ...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

In [55]:
final_pipe.predict(X_val)

array([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1])

In [56]:
import pickle

# Export pipeline as pickle file
with open("pipeline_8.pkl", "wb") as file:
    pickle.dump(final_pipe, file)